In [91]:
import pandas as pd

In [92]:
df = pd.read_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/df_web_experiment_cleand_steps")

In [93]:
df

,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,Variation
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:27:07,Test
1,1,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:26:51,Test
2,2,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:19:22,Test
3,3,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:13,Test
4,4,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:18:04,Test
...,...,...,...,...,...,...,...
317230,417650,1574008,117364417_77840596075,528720790_71583064618_169151,1,2017-05-06 23:43:27,Test
317231,417651,2908510,814969699_90652851448,562606085_36368381773_92090,1,2017-05-10 22:57:17,Control
317232,417652,2908510,814969699_90652851448,562606085_36368381773_92090,3,2017-05-10 22:56:31,Control
317233,417653,2908510,814969699_90652851448,562606085_36368381773_92090,2,2017-05-10 22:56:23,Control


In [94]:
# Creating column for repeated step by visit_id, and creating another column to display the step name if repeated
df['repeated_step'] = df.duplicated(subset=['visitor_id', 'process_step'], keep=False).astype(int)
df['step_repeated'] = df['process_step'].where(df['repeated_step'] == 1)

# replacing null values in step_repeated with 0
df['step_repeated'] = df['step_repeated'].fillna(0).astype(int)

# adding a boolean column to indicate if an error occured (i.e., if a step was repeated)
df['error_occurred'] = df['repeated_step'].apply(lambda x: True if x == 1 else False)


df



,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,Variation,repeated_step,step_repeated,error_occurred
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:27:07,Test,1,4,True
1,1,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:26:51,Test,1,3,True
2,2,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:19:22,Test,1,4,True
3,3,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:13,Test,1,3,True
4,4,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:18:04,Test,1,4,True
...,...,...,...,...,...,...,...,...,...,...
317230,417650,1574008,117364417_77840596075,528720790_71583064618_169151,1,2017-05-06 23:43:27,Test,0,0,False
317231,417651,2908510,814969699_90652851448,562606085_36368381773_92090,1,2017-05-10 22:57:17,Control,1,1,True
317232,417652,2908510,814969699_90652851448,562606085_36368381773_92090,3,2017-05-10 22:56:31,Control,0,0,False
317233,417653,2908510,814969699_90652851448,562606085_36368381773_92090,2,2017-05-10 22:56:23,Control,0,0,False


In [95]:
# Ensure date_time is datetime
df['date_time'] = pd.to_datetime(df['date_time'])

# Find the row index of the maximum date_time per visit_id
idx = df.groupby('visit_id')['date_time'].idxmax()

# Set all last_steps to empty by default
df['last_step'] = ""

# Assign process_step as last_step only for the last row of each visit
df.loc[idx, 'last_step'] = df.loc[idx, 'process_step']

In [96]:
# Calculate the total time spent in each visit
visit_time_spans = df.groupby('visit_id')['date_time'].agg(lambda x: x.max() - x.min())

# Assign each row the time spent for its visit_id
df['time_spent'] = df['visit_id'].map(visit_time_spans).astype(str).str.split(" ").str[2]

df = df[df['last_step'] != ""]

df

,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,Variation,repeated_step,step_repeated,error_occurred,last_step,time_spent
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:27:07,Test,1,4,True,4,00:10:45
8,8,8320017,39393514_33118319366,960651974_70596002104_312201,5,2017-04-05 13:10:05,Test,0,0,False,5,00:01:59
13,13,4033851,762728880_76361333336,949661017_22392791362_127391,5,2017-04-05 12:29:03,Control,1,5,True,5,00:25:03
24,24,1982004,618934751_10082078917,132494692_73738638511_70823,5,2017-04-17 12:02:42,Test,0,0,False,5,00:03:00
29,29,9294070,489274391_63794562906,116640698_27328181095_188045,1,2017-04-17 11:16:28,Control,1,1,True,1,00:00:15
...,...,...,...,...,...,...,...,...,...,...,...,...
317215,417634,6426623,27047012_3402532492,685634090_42070675506_389171,5,2017-05-19 05:40:14,Control,0,0,False,5,00:03:01
317220,417639,433098,5505424_50616523266,773565250_67577573147_389841,5,2017-05-13 15:33:04,Control,0,0,False,5,00:02:17
317225,417644,7049106,710348688_12741653529,412278907_8949064401_171038,1,2017-05-20 23:07:09,Test,1,1,True,1,00:00:00
317226,417646,1574008,117364417_77840596075,528720790_71583064618_169151,5,2017-05-06 23:51:05,Test,0,0,False,5,00:07:38


In [97]:
# Create function to change type of time_spent to numeric
def convert_time_spent(df):
    df['time_spent'] = pd.to_datetime(df['time_spent'], format='%H:%M:%S').dt.time
    df['time_spent'] = df['time_spent'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    return df

df = convert_time_spent(df)

/var/folders/wc/048rx86x7bqbzmjn7nr46j_80000gn/T/ipykernel_8468/2256036968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_spent'] = pd.to_datetime(df['time_spent'], format='%H:%M:%S').dt.time
/var/folders/wc/048rx86x7bqbzmjn7nr46j_80000gn/T/ipykernel_8468/2256036968.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_spent'] = df['time_spent'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)


In [98]:
# Add column to determine if last step "5" was completed or not
df['confirmed'] = df['process_step'].apply(lambda x: True if x == 5 else False)



df = df.drop(columns=['Unnamed: 0'])


/var/folders/wc/048rx86x7bqbzmjn7nr46j_80000gn/T/ipykernel_8468/3854972091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['confirmed'] = df['process_step'].apply(lambda x: True if x == 5 else False)


In [99]:
# Keeping the last occurence of visit_id to avoid duplication in analysis
df = df.sort_values(by=['date_time']).drop_duplicates(subset=['visit_id'], keep='last')

In [100]:
df

,client_id,visitor_id,visit_id,process_step,date_time,Variation,repeated_step,step_repeated,error_occurred,last_step,time_spent,confirmed
34567,7179755,167765295_97487764427,264484508_5982901710_928530,5,2017-03-15 00:26:24,Control,0,0,False,5,416,True
77851,4192640,692067844_75217592829,706721307_85347845958_18583,1,2017-03-15 00:44:38,Test,1,1,True,1,75,False
102703,6752370,258848572_66112715827,147442660_10728929690_338280,1,2017-03-15 05:43:13,Test,1,1,True,1,338,False
204715,9584408,748244138_48778380454,484298588_81471639218_981974,5,2017-03-15 06:16:08,Control,0,0,False,5,764,True
170723,2685910,321566510_66607009808,538067801_99657933298_140950,1,2017-03-15 07:13:47,Test,1,1,True,1,60,False
...,...,...,...,...,...,...,...,...,...,...,...,...
301099,6800679,185313147_57950282409,401866349_68590569683_395732,1,2017-06-20 23:14:06,Test,0,0,False,1,0,False
243357,1545935,976873968_42239039722,10056419_61656665964_803695,5,2017-06-20 23:21:23,Test,0,0,False,5,457,True
287010,291330,992344309_24884468623,91879336_25502670457_529422,2,2017-06-20 23:21:41,Control,0,0,False,2,7,False
314908,1558312,397562195_28573798537,175513303_32086468382_224926,5,2017-06-20 23:26:00,Control,0,0,False,5,745,True


In [101]:
df.to_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/cleaned/df_web_experiment_a.csv", index=False)

In [102]:
df = pd.read_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/df_web_experiment_cleand_steps")

# Creating column for repeated step by visit_id, and creating another column to display the step name if repeated
df['repeated_step'] = df.duplicated(subset=['visitor_id', 'process_step'], keep=False).astype(int)
df['step_repeated'] = df['process_step'].where(df['repeated_step'] == 1)

# replacing null values in step_repeated with 0
df['step_repeated'] = df['step_repeated'].fillna(0).astype(int)

df.drop(columns=['Unnamed: 0'], inplace=True)
df.to_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/cleaned/df_web_experiment_steps.csv", index=False)

In [103]:
df

,client_id,visitor_id,visit_id,process_step,date_time,Variation,repeated_step,step_repeated
0,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:27:07,Test,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:26:51,Test,1,3
2,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:19:22,Test,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:13,Test,1,3
4,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:18:04,Test,1,4
...,...,...,...,...,...,...,...,...
317230,1574008,117364417_77840596075,528720790_71583064618_169151,1,2017-05-06 23:43:27,Test,0,0
317231,2908510,814969699_90652851448,562606085_36368381773_92090,1,2017-05-10 22:57:17,Control,1,1
317232,2908510,814969699_90652851448,562606085_36368381773_92090,3,2017-05-10 22:56:31,Control,0,0
317233,2908510,814969699_90652851448,562606085_36368381773_92090,2,2017-05-10 22:56:23,Control,0,0


In [104]:
df["visitor_id"].nunique()

56011

In [105]:
df['visitor_id'].nunique()

56011